In [3]:
import pandas as pd
import nltk
import os

In [4]:
os.getcwd()

'/Users/chasedawson/dev/sdad/amsoldier/src'

In [5]:
filename = "../data/s144_pos_topics.csv"

In [6]:
df = pd.read_csv(filename, index_col = 0)
df.head()

,id,subject_id,image_name,document_type,q85,sentiment,score,topic
4,3,15901768,2521127-37-0004.jpg,A document that contains Q.85,concerning question i havent the slightest ide...,POSITIVE,0.997207,1
6,5,15901787,2521127-37-0006.jpg,A document that contains Q.85,the army would be swell for a certain amount o...,POSITIVE,0.890350,0
11,10,15901828,2521127-37-0011.jpg,A document that contains Q.85,the army is in so bad but i am reader to be ho...,POSITIVE,0.927433,0
12,11,15901835,2521127-37-0012.jpg,A document that contains Q.85,may god bless us all,POSITIVE,0.999380,1
13,12,15901840,2521127-37-0013.jpg,A document that contains Q.85,there is one thing. i do believe that the colo...,POSITIVE,0.998074,-1


In [123]:
word_counts = pd.read_csv("../data/word_counts_over_time_intervals_v2.csv", index_col = 1)
word_counts = word_counts.drop(columns=["Unnamed: 0"])
word_counts.head()

,count,interval_id
word,,
vote,86.0,0.0
#stopthest,48.0,0.0
trump,40.0,0.0
#wwg1wga,39.0,0.0
#americafirst,39.0,0.0


In [124]:
word_counts.shape

(1087184, 2)

In [136]:
round(0.4566778, 4)

0.4567

In [137]:
def make_links(interval):
    data = df[df.interval_id == interval]
    links = []
    for i, row in data.iterrows():
        if i % 2 == 1:
            links.append({'source': row['item1'], 'target': row['item2'], 'count': str(int(row['count'])), 'phi': str(round(row['phi'], 3))})
    return links

In [138]:
intervals = list(df.interval_id.value_counts().index.sort_values())
intervals[:5]

[0.0, 1.0, 2.0, 3.0, 4.0]

In [139]:
links_over_intervals = {}
for interval in intervals:
    links_over_intervals[str(int(interval))] = make_links(interval)

In [140]:
pd.DataFrame(links_over_intervals["17"]).head()

,source,target,count,phi
0,america,vote,215,1.0
1,trump,vote,192,0.69
2,vote,go,177,1.0
3,vote,light,177,1.0
4,vote,expos,177,1.0


In [37]:
!pip install python-louvain

     |████████████████████████████████| 204 kB 5.7 MB/s eta 0:00:01
  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)
  Created wheel for python-louvain: filename=python_louvain-0.15-py3-none-any.whl size=9401 sha256=c18c6ee897a1ecc398a6919c13af59357849bc7a34a135aef289a9de01982c4c
  Stored in directory: /Users/chasedawson/Library/Caches/pip/wheels/c8/68/21/d04741956802581119d875a0d9fce6895e8f9ffae71223f3e3
Successfully built python-louvain


In [39]:
# get communities

In [141]:
# add indices into word counts
index = []
for interval in intervals:
    data = word_counts[word_counts.interval_id == interval]
    for i in range(len(data)):
        index.append(i)
word_counts['index'] = index
word_counts.head()

,count,interval_id,index
word,,,
vote,86.0,0.0,0
#stopthest,48.0,0.0,1
trump,40.0,0.0,2
#wwg1wga,39.0,0.0,3
#americafirst,39.0,0.0,4


In [142]:
import community as community_louvain
import networkx as nx

def get_edges(interval_id):
    data = df[df.interval_id == interval_id]
    wc = word_counts[word_counts.interval_id == interval_id]
    edges = []
    for i, row in data.iterrows():
        edge = (wc.loc[row['item1']]['index'], wc.loc[row['item2']]['index'])
        edges.append(edge)
    return edges   

In [143]:
# add communtiy assignments to word counts
import numpy as np
word_counts["community"] = np.nan
for interval in intervals:
    edges = get_edges(interval)
    G = nx.Graph()
    G.add_edges_from(edges100)
    partition = community_louvain.best_partition(G)
    for key in partition:
        word_counts.loc[(word_counts.interval_id == interval) & (word_counts['index'] == key), 'community'] = partition[key]

In [144]:
def make_nodes(interval_id):
    data = df[df.interval_id == interval_id]
    wc = word_counts[word_counts.interval_id == interval_id]
    nmap = {}
    nodes = []
    for i in range(len(data)):
        # add item1
        nodes = add_item(data.iloc[i]['item1'], wc, nmap, nodes)
        
        # add item2
        nodes = add_item(data.iloc[i]['item2'], wc, nmap, nodes)
        
    return nodes
        
                        
def add_item(word, wc, nmap, nodes):
    if word not in nmap:
        nmap[word] = word
        count = wc.loc[word]['count']
        community = wc.loc[word]['community']
        if np.isnan(community):
            community = -1
        nodes.append({'id': word, 'label': word, 'count': str(int(count)), 'louvain': str(int(community))})
    return nodes

In [146]:
nodes_over_intervals = {}
for interval in intervals:
    nodes_over_intervals[str(int(interval))] = make_nodes(interval)

In [147]:
data_over_intervals = {'nodes': nodes_over_intervals, 'links': links_over_intervals}

In [158]:
intervals[:5]

[0.0, 1.0, 2.0, 3.0, 4.0]

In [160]:
# filter out hashtags in nodes
for interval in intervals:
    nodes = data_over_intervals['nodes'][str(int(interval))]
    for node in nodes.copy():
        if node['id'][0] == '#':
            nodes.remove(node)

In [166]:
for interval in intervals:
    links = data_over_intervals['links'][str(int(interval))]
    for link in links.copy():
        if link['source'][0] == '#' or link['target'][0] == '#':
            links.remove(link)

In [167]:
import json
with open('../data/cor_no_hashtags.json', 'w') as fp:
    json.dump(data_over_intervals, fp)

In [87]:
# recode intervals
for interval in intervals:
    new_key = str(int(interval))
    data_over_intervals['nodes'][new_key] = data_over_intervals['nodes'].pop(interval)
    data_over_intervals['links'][new_key] = data_over_intervals['links'].pop(interval)